In [1]:
import numpy as np
import random
import copy
from numba import jit
import time
import cProfile

In [2]:
class World:
    """
    The World class encapsulates information about the computational domain.
    
    Attributes
    ----------
    <to be written later>
    ----------
    
    """
    
    def __init__(self, ni, nj, nk):
        """
        Initializes the World object with the grid parameters

        Parameters
        ----------
        ni : int
            number of grid points along x coordinate
        nj : int
            number of grid points along y coordinate
        nk : int
            number of grid points along z coordinate
        ----------
        
        """
        
        self.ni = ni
        self.nj = nj
        self.nk = nk
        
        self.nn = np.zeros(3)
        
        self.nn[0] = self.ni
        self.nn[1] = self.nj
        self.nn[2] = self.nk
        
        self.x0 = np.zeros(3)
        self.dh = np.zeros(3)
        self.xm = np.zeros(3)
        self.xc = np.zeros(3)
        
        self.EPS_0 = 8.85418782e-12
        self.QE    = 1.602176565e-19;
        self.AMU   = 1.660538921e-27
        self.ME    = 9.10938215e-31;
        self.K     = 1.380648e-23;
        self.EvToK = self.QE/self.K;
        
        self.phi        = np.zeros((self.ni, self.nj, self.nk))
        self.phi_new    = np.zeros((self.ni, self.nj, self.nk))
        self.R          = np.zeros((self.ni, self.nj, self.nk))
        self.rho        = np.zeros((self.ni, self.nj, self.nk))
        self.node_vol   = np.zeros((self.ni, self.nj, self.nk))
        self.ef         = np.zeros((self.ni, self.nj, self.nk, 3))
    
    def setTime(self, dt, num_ts):
        self.dt     = dt
        self.num_ts = num_ts
        
    def setExtents(self, x1, y1, z1, x2, y2, z2):
        
        """
        Set mesh extents and compute grid spacing.

        Parameters
        ----------
        x1 : float
            x-coordinate of world origin
        y1 : float
            y-coordinate of world origin
        z1 : float
            z-coordinate of world origin
        x2 : float
            x-coordinate of world max bound
        y2 : float
            y-coordinate of world max bound
        z2 : float
            z-coordinate of world max bound
        ----------
        
        """
        
        self.x0[0] = x1
        self.x0[1] = y1
        self.x0[2] = z1
        
        self.xm[0] = x2
        self.xm[1] = y2
        self.xm[2] = z2
        
        for i in range(3):
            self.dh[i] = (self.xm[i] - self.x0[i]) / (self.nn[i] - 1)
            self.xc[i] = 0.5*(self.x0[i] + self.xm[i]) 
        
        self.computeNodeVolumes()
    
    def computeDebyeLength(self, Te, ne):
        """
        Compute the Debye length.

        Parameters
        ----------
        Te : float
            electron temperature, K
        ne : float
            number density, m3
        ----------
        
        """
        
        return np.sqrt(self.EPS_0*self.K*Te/(ne*self.QE**2))
    
    
    def potentialSolver(self, max_it, tol):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        # solve potential
        for it in np.arange(1,max_it+1):
            for i in np.arange(1,self.ni-1):
                for j in np.arange(1,self.nj-1):
                    for k in np.arange(1,self.nk-1):
                        #standard internal open node
                        phi_new = (self.rho[i][j][k]/self.EPS_0 +\
                                        dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +\
                                        dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +\
                                        dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1]))/(2*dx2+2*dy2+2*dz2)
                        
                        # sucessive over relaxation  
                        self.phi[i,j,k] = self.phi[i,j,k] + 1.4*(phi_new - self.phi[i][j][k])
        

            #check for convergence*/
            if it%25==0:
                sum = 0;
                for i in np.arange(1,self.ni-1):
                    for j in np.arange(1,self.nj-1):
                        for k in np.arange(1,self.nk-1):

                            R = -self.phi[i][j][k]*(2*dx2+2*dy2+2*dz2) +\
                            self.rho[i][j][k]/self.EPS_0 +\
                            dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +\
                            dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +\
                            dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1])

                            sum += R*R;


                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                #print("iter: "+str(it)+", L2 = "+str(L2))
                if (L2<tol):
                    converged = True
                    break
                    
        if (converged==False):
            print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged
    
    def potentialSolver22(self, max_it, tol):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        
        # solve potential
        for it in np.arange(1,max_it+1):
            """
            for i in np.arange(1,self.ni-1):
                for j in np.arange(1,self.nj-1):
                    for k in np.arange(1,self.nk-1):
            """            
            """
            #standard internal open node
                        phi_new = (self.rho[i][j][k]/self.EPS_0 +\
                                        dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +\
                                        dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +\
                                        dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1]))/(2*dx2+2*dy2+2*dz2)
                                        
            # sucessive over relaxation  
                        self.phi[i,j,k] = self.phi[i,j,k] + 1.4*(phi_new - self.phi[i][j][k])
        
            """            
            #standard internal open node
            self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                            (self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                            dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1])+\
                            dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1])+\
                            dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk]))/(2*dx2+2*dy2+2*dz2)
            
            """
            # sucessive over relaxation  
            self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1] = self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1] +\
                                                        1.8*(self.phi_new[1:self.ni-1,1:self.nj-1,1:self.nk-1] - \
                                                        self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1])
            """
            
            #check for convergence*/
            if it%25==0:
                sum = 0;
                """
                for i in np.arange(1,self.ni-1):
                    for j in np.arange(1,self.nj-1):
                        for k in np.arange(1,self.nk-1):
                """

                self.R[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                -self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1]*(2*dx2+2*dy2+2*dz2) +\
                self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1]) +\
                dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1]) +\
                dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk])

                sum = np.sum(self.R**2)

                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                #print("iter: "+str(it)+", L2 = "+str(L2))
                if (L2<tol):
                    converged = True
                    break

        if (converged==False):
            print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged
    
    
    def potentialSolver4(self, w, max_it, tol):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        # Step 1: create *integer* array the same size as u 
        x = np.zeros_like(self.phi,dtype=np.int)

        # Step 2: populate all non-boundary cells with running numbers from 1 to (n-2)^2
        x[1:-1,1:-1,1:-1] = np.arange(1,(self.ni-2)*(self.nj-2)*(self.nk-2)+1).reshape(self.ni-2,self.nj-2,self.nk-2)

        # Step 3: get indices of even (red) and odd (black) points
        ir, jr, kr = np.where((x>0) & (x%2 == 0)) # indices of red pts = indices of even numbers
        ib, jb, kb = np.where((x>0) & (x%2 == 1)) # indices of black pts = indices of odd numbers


        
        # solve potential
        for it in np.arange(1,max_it+1):
            """
            for i in np.arange(1,self.ni-1):
                for j in np.arange(1,self.nj-1):
                    for k in np.arange(1,self.nk-1):
            """            
            """
            #standard internal open node
                        phi_new = (self.rho[i][j][k]/self.EPS_0 +\
                                        dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +\
                                        dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +\
                                        dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1]))/(2*dx2+2*dy2+2*dz2)
                                        
            # sucessive over relaxation  
                        self.phi[i,j,k] = self.phi[i,j,k] + 1.4*(phi_new - self.phi[i][j][k])
        
            """ 
            
          
            #standard internal open node
            self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                            (self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                            dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1])+\
                            dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1])+\
                            dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk]))/(2*dx2+2*dy2+2*dz2)
            
           
            
            

            #check for convergence*/
            if it%25==0:
                sum = 0;
                """
                for i in np.arange(1,self.ni-1):
                    for j in np.arange(1,self.nj-1):
                        for k in np.arange(1,self.nk-1):
                """

                self.R[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                -self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1]*(2*dx2+2*dy2+2*dz2) +\
                self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1]) +\
                dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1]) +\
                dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk])

                sum = np.sum(self.R**2)

                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                #print("iter: "+str(it)+", L2 = "+str(L2))
                if (L2<tol):
                    converged = True
                    break

        if (converged==False):
            print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged

    
    def potentialSolver5(self, w, max_it, tol):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        # Step 1: create *integer* array the same size as u 
        x = np.zeros_like(self.phi,dtype=np.int)

        # Step 2: populate all non-boundary cells with running numbers from 1 to (n-2)^2
        x[1:-1,1:-1,1:-1] = np.arange(1,(self.ni-2)*(self.nj-2)*(self.nk-2)+1).reshape(self.ni-2,self.nj-2,self.nk-2)

        # Step 3: get indices of even (red) and odd (black) points
        ir, jr, kr = np.where((x>0) & (x%2 == 0)) # indices of red pts = indices of even numbers
        ib, jb, kb = np.where((x>0) & (x%2 == 1)) # indices of black pts = indices of odd numbers


        
        # solve potential
        for it in np.arange(1,max_it+1):
            """
            for i in np.arange(1,self.ni-1):
                for j in np.arange(1,self.nj-1):
                    for k in np.arange(1,self.nk-1):
            """            
            """
            #standard internal open node
                        phi_new = (self.rho[i][j][k]/self.EPS_0 +\
                                        dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +\
                                        dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +\
                                        dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1]))/(2*dx2+2*dy2+2*dz2)
                                        
            # sucessive over relaxation  
                        self.phi[i,j,k] = self.phi[i,j,k] + 1.4*(phi_new - self.phi[i][j][k])
        
            """ 
            
            """
            #standard internal open node
            self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                            (self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                            dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1])+\
                            dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1])+\
                            dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk]))/(2*dx2+2*dy2+2*dz2)
            
            """
            
            # Red point update
            self.phi[ir,jr,kr] = (1-w)*self.phi[ir,jr,kr] + (1.0/6.0)*w*(self.phi[ir+1,jr,kr] + self.phi[ir-1,jr,kr] +\
                                                                    self.phi[ir,jr+1,kr] + self.phi[ir,jr-1,kr] +\
                                                                    self.phi[ir,jr,kr+1] + self.phi[ir,jr,kr-1] +\
                                                                    (self.rho[ir,jr,kr]/self.EPS_0)*(self.dh[0]*self.dh[1]))

            # Black point update
            self.phi[ib,jb,kb] = (1-w)*self.phi[ib,jb,kb] + (1.0/6.0)*w*(self.phi[ib+1,jb,kb] + self.phi[ib-1,jb,kb] +\
                                                                    self.phi[ib,jb+1,kb] + self.phi[ib,jb-1,kb] +\
                                                                    self.phi[ib,jb,kb+1] + self.phi[ib,jb,kb-1] +\
                                                                    (self.rho[ib,jb,kb]/self.EPS_0)*(self.dh[0]*self.dh[1]))
            
            """
            # Red point update
            self.phi[ir,jr,kr] = (1-w)*self.phi[ir,jr,kr] + w*((self.phi[ir+1,jr,kr] + self.phi[ir-1,jr,kr])*dx2 +\
                                                                         (self.phi[ir,jr+1,kr] + self.phi[ir,jr-1,kr])*dy2 +\
                                                                         (self.phi[ir,jr,kr+1] + self.phi[ir,jr,kr-1])*dz2 +\
                                                                         (self.rho[ir,jr,kr]/self.EPS_0))/(2*dx2+2*dy2+2*dz2)

            # Black point update
            self.phi[ib,jb,kb] = (1-w)*self.phi[ib,jb,kb] + w*((self.phi[ib+1,jb,kb] + self.phi[ib-1,jb,kb])*dx2 +\
                                                                         (self.phi[ib,jb+1,kb] + self.phi[ib,jb-1,kb])*dy2 +\
                                                                         (self.phi[ib,jb,kb+1] + self.phi[ib,jb,kb-1])*dz2 +\
                                                                         (self.rho[ib,jb,kb]/self.EPS_0))/(2*dx2+2*dy2+2*dz2)
           
            """
            #check for convergence*/
            if it%25==0:
                sum = 0;
                """
                for i in np.arange(1,self.ni-1):
                    for j in np.arange(1,self.nj-1):
                        for k in np.arange(1,self.nk-1):
                """

                self.R[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                -self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1]*(2*dx2+2*dy2+2*dz2) +\
                self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1]) +\
                dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1]) +\
                dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk])

                sum = np.sum(self.R**2)

                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                #print("iter: "+str(it)+", L2 = "+str(L2))
                if (L2<tol):
                    converged = True
                    break

        #if (converged==False):
        #    print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged

    
    def efSolver(self):
        """
        Compute the electric field from potential function.

        Parameters
        ----------
        
        ----------
        
        """
        dx = self.dh[0] # dx
        dy = self.dh[1] # dy
        dz = self.dh[2] # dz
        
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):

                    #x-component#
                    if i==0: 
                        # forward
                        self.ef[i][j][k][0] = -(-3*self.phi[i][j][k]+\
                                               4*self.phi[i+1][j][k]-\
                                               self.phi[i+2][j][k])/(2*dx)
                    elif i==self.ni-1:  
                        # backward
                        self.ef[i][j][k][0] = -(self.phi[i-2][j][k]-\
                                               4*self.phi[i-1][j][k]+\
                                               3*self.phi[i][j][k])/(2*dx)
                    else: 
                        #central
                        self.ef[i][j][k][0] = -(self.phi[i+1][j][k] - \
                                                self.phi[i-1][j][k])/(2*dx)

                    #y-component
                    if j==0:
                        self.ef[i][j][k][1] = -(-3*self.phi[i][j][k] + \
                                                4*self.phi[i][j+1][k]-\
                                                self.phi[i][j+2][k])/(2*dy)
                    elif j==self.nj-1:
                        self.ef[i][j][k][1] = -(self.phi[i][j-2][k] - \
                                                4*self.phi[i][j-1][k] +\
                                                3*self.phi[i][j][k])/(2*dy)
                    else:
                         self.ef[i][j][k][1] = -(self.phi[i][j+1][k] - \
                                                 self.phi[i][j-1][k])/(2*dy)

                    #z-component
                    if k==0:
                        self.ef[i][j][k][2] = -(-3*self.phi[i][j][k] + \
                                                4*self.phi[i][j][k+1]-
                                                self.phi[i][j][k+2])/(2*dz)
                    elif k==self.nk-1:
                        self.ef[i][j][k][2] = -(self.phi[i][j][k-2] - \
                                                4*self.phi[i][j][k-1]    + \
                                                3*self.phi[i][j][k])/(2*dz)
                    else:
                        self.ef[i][j][k][2] = -(self.phi[i][j][k+1] - \
                                                self.phi[i][j][k-1])/(2*dz)
    
    def efSolver2(self):
        """
        Compute the electric field from potential function.

        Parameters
        ----------
        
        ----------
        
        """
        dx = self.dh[0] # dx
        dy = self.dh[1] # dy
        dz = self.dh[2] # dz
        
        """
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):
        """

        ##x-component#
        #if i==0: 
        #x-component#
        """
                    if i==0: 
                        # forward
                        self.ef[i][j][k][0] = -(-3*self.phi[i][j][k]+\
                                               4*self.phi[i+1][j][k]-\
                                               self.phi[i+2][j][k])/(2*dx)
        """
                    
        # forward
        self.ef[0,0:self.nj,0:self.nk,0] = -(-3*self.phi[0,0:self.nj,0:self.nk]+\
                               4*self.phi[1,0:self.nj,0:self.nk]-\
                               self.phi[2,0:self.nj,0:self.nk])/(2*dx)
        
        #elif i==self.ni-1:  
        """
        elif i==self.ni-1:  
                        # backward
                        self.ef[i][j][k][0] = -(self.phi[i-2][j][k]-\
                                               4*self.phi[i-1][j][k]+\
                                               3*self.phi[i][j][k])/(2*dx)
        """           
        # backward
        self.ef[self.ni-1,0:self.nj,0:self.nk,0] = -(self.phi[self.ni-3,0:self.nj,0:self.nk]-\
                                   4*self.phi[self.ni-2,0:self.nj,0:self.nk]+\
                                   3*self.phi[self.ni-1,0:self.nj,0:self.nk])/(2*dx)
        """
        else: 
            #central
            self.ef[i][j][k][0] = -(self.phi[i+1][j][k] - \
                                    self.phi[i-1][j][k])/(2*dx)
        """ 
        #central
        self.ef[1:self.ni-1,0:self.nj,0:self.nk,0] = -(self.phi[2:self.ni,0:self.nj,0:self.nk] - \
                                self.phi[0:self.ni-2,0:self.nj,0:self.nk])/(2*dx)


        #y-component
        #if j==0:
        """
        if j==0:
                        self.ef[i][j][k][1] = -(-3*self.phi[i][j][k] + \
                                                4*self.phi[i][j+1][k]-\
                                                self.phi[i][j+2][k])/(2*dy)
                    
        """
        self.ef[0:self.ni,0,0:self.nk,1] = -(-3*self.phi[0:self.ni,0,0:self.nk] + \
                                    4*self.phi[0:self.ni,1,0:self.nk]-\
                                    self.phi[0:self.ni,2,0:self.nk])/(2*dy)
        #elif j==self.nj-1:
        """
        elif j==self.nj-1:
                        self.ef[i][j][k][1] = -(self.phi[i][j-2][k] - \
                                                4*self.phi[i][j-1][k] +\
                                                3*self.phi[i][j][k])/(2*dy)
                    
        """
        self.ef[0:self.ni,self.nj-1,0:self.nk,1] = -(self.phi[0:self.ni,self.nj-3,0:self.nk] - \
                                    4*self.phi[0:self.ni,self.nj-2,0:self.nk] +\
                                    3*self.phi[0:self.ni,self.nj-1,0:self.nk])/(2*dy)
        #else:
        """
        else:
                         self.ef[i][j][k][1] = -(self.phi[i][j+1][k] - \
                                                 self.phi[i][j-1][k])/(2*dy)

        """
        self.ef[0:self.ni,1:self.nj-1,0:self.nk,1] = -(self.phi[0:self.ni,2:self.nj,0:self.nk] - \
                                     self.phi[0:self.ni,0:self.nj-2,0:self.nk])/(2*dy)

        #z-component
        '''
        if k==0:
            self.ef[i][j][k][2] = -(-3*self.phi[i][j][k] + \
                                    4*self.phi[i][j][k+1]-
                                    self.phi[i][j][k+2])/(2*dz)
            
        '''
        #z-component
        #if k==0:
        self.ef[0:self.ni,0:self.nj,0,2] = -(-3*self.phi[0:self.ni,0:self.nj,0] + \
                                4*self.phi[0:self.ni,0:self.nj,1]-
                                self.phi[0:self.ni,0:self.nj,2])/(2*dz)

        """
        elif k==self.nk-1:
            self.ef[i][j][k][2] = -(self.phi[i][j][k-2] - \
                                    4*self.phi[i][j][k-1]    + \
                                    3*self.phi[i][j][k])/(2*dz)
        """
        
        #elif k==self.nk-1:
        self.ef[0:self.ni,0:self.nj,self.nk-1,2] = -(self.phi[0:self.ni,0:self.nj,self.nk-3] - \
                                    4*self.phi[0:self.ni,0:self.nj,self.nk-2]    + \
                                    3*self.phi[0:self.ni,0:self.nj,self.nk-1])/(2*dz) 
        """
        else:
            self.ef[i][j][k][2] = -(self.phi[i][j][k+1] - \
                                    self.phi[i][j][k-1])/(2*dz)
        """
        #else:
        self.ef[0:self.ni,0:self.nj,1:self.nk-1,2] = -(self.phi[0:self.ni,0:self.nj,2:self.nk] - \
                                    self.phi[0:self.ni,0:self.nj,0:self.nk-2])/(2*dz)
        
        
    def computeNodeVolumes(self): 
        """
        Compute the node volumes.
        Parameters
        ----------
        
        ----------
        
        """
        for i in np.arange(0,self.ni):
            for j in np.arange(0,self.nj):
                for k in np.arange(0,self.nk):
        
                    V = self.dh[0]*self.dh[1]*self.dh[2]
                    if (i==0 or i==self.ni-1): V*=0.5
                    if (j==0 or j==self.nj-1): V*=0.5
                    if (k==0 or k==self.nk-1): V*=0.5
                    
                    self.node_vol[i][j][k] = V
                    
    def XtoL(self, x):
        """
        Determine which cell a particle at position vector 
        x belongs to.
        
        Parameters
        ----------
        x : numpy.ndarray
            position vector 
        ----------
        """
        lc = np.zeros(3)
        
        lc[0] = (x[0]-self.x0[0])/self.dh[0];
        lc[1] = (x[1]-self.x0[1])/self.dh[1];
        lc[2] = (x[2]-self.x0[2])/self.dh[2];
        
        return lc
    
    def addSpeciesList(self, speciesList):
        self.speciesList = speciesList
    
    def computeChargeDensity(self):
        """
        Compute the charge density.
        
        Parameters
        ----------
     
        ----------
        """
        
        self.rho = np.zeros((self.ni, self.nj, self.nk))
        
        for species in self.speciesList:
            if species.charge!=0:
                self.rho += species.charge*species.den         

In [3]:
class Particle:
    """
    The Particle class encapsulates information about the particles 
    used in the simulation.
    
    Attributes
    ----------
    
    ----------
    
    """
    
    def __init__(self, pos, vel, mpw):
        """
        Initializes the Particle object with the position, 
        speed, and macrparticle weight.

        Parameters
        ----------
        pos : numpy.ndarray
            particle position vector
        vel : numpy.ndarray
            particle velocity vector
        mpw : float
            macroparticle weight  
        ----------
        
        """
        
        self.pos = pos
        self.vel = vel
        self.mpw = mpw
        

class Species:
    """
    The Species class encapsulates information about the species 
    used in the simulation.
    
    Attributes
    ----------
    
    ----------
    
    """
    def __init__(self, name, mass, charge, worldObj):
        """
        Initializes the Species object with the name, mass,
        charge.

        Parameters
        ----------
        name : str
            species name
        mass : float
            species mass
        charge : float
            species charge
        mpw : float
            macroparticle weight  
        ----------

        """
        
        self.particleList = []
        
        self.name   = name
        self.mass   = mass
        self.charge = charge
        
        self.den    = np.zeros((worldObj.ni, worldObj.nj, worldObj.nk))
        
        self.worldObj = worldObj
    
    def addParticle(self, pos, vel, mpw):
        """random.random()
        add a particle to particleList

         Parameters
        ----------
        pos : numpy.ndarray
            particle position vector
        vel : numpy.ndarray
            particle velocity vector
        mpw : float
            macroparticle weight  
        ----------
        
        """
        #get logical coordinate of particle's position
        lc = self.worldObj.XtoL(pos)

        #electric field at particle position
        ef_part = self.gather_ef(lc)

        #rewind velocity back by 0.5*dt*ef
        vel -=  self.charge/self.mass*ef_part*(0.5*self.worldObj.dt);
        
        #add particle to list
        self.particleList.append(Particle(pos, vel, mpw))
        
        
    def loadParticlesBox(self, x1, x2, num_den, num_mp):
        """
        loads randomly distributed particles in a x1-x2 box 
        representing num_den number density

        Parameters
        ----------
        x1 : numpy.ndarray
            origin of bounding box
        x2 : numpy.ndarray
            max. bound corner of box
        num_den : float
            number density
        num_mp  : number of macroparticles
        ----------

        """
        box_vol  = (x2[0]-x1[0])*(x2[1]-x1[1])*(x2[2]-x1[2]) # box vol.
        num_real = num_den * box_vol;                   #number of real particles
        mpw      = num_real/num_mp;                     # macroparticle weight
        
        self.box_vol = box_vol
        self.num_real= num_real
        self.mpw     = mpw

        #load particles on an equally spaced grid
        for p in np.arange(0,num_mp+1):
            # sample random position
            pos = np.zeros(3)
            vel = np.zeros(3)

            #rnd = random.random()

            pos[0] = x1[0] + random.random()*(x2[0]-x1[0]);
            pos[1] = x1[1] + random.random()*(x2[1]-x1[1]);
            pos[2] = x1[2] + random.random()*(x2[2]-x1[2]);

            #set initial velocity
            vel[0] = 0.0
            vel[1] = 0.0
            vel[2] = 0.0

            self.addParticle(pos,vel,mpw); # add new particle
            
    def loadParticlesBoxQS(self, x1, x2, num_den, num_mp):
        """
        loads randomly distributed particles in a x1-x2 box 
        representing num_den number density

        Parameters
        ----------
        x1 : numpy.ndarray
            origin of bounding box
        x2 : numpy.ndarray
            max. bound corner of box
        num_den : float
            number density
        num_mp  : numpy.ndarray
        ----------

        """
        box_vol  = (x2[0]-x1[0])*(x2[1]-x1[1])*(x2[2]-x1[2]) # box vol.
        num_real = num_den * box_vol;                   #number of real particles
        num_mp_tot = (num_mp[0]-1)*(num_mp[1]-1)*(num_mp[2]-1)
        mpw      = num_real/num_mp_tot;                     # macroparticle weight
        
        self.box_vol = box_vol
        self.num_real= num_real
        self.mpw     = mpw
        
        di = (x2[0]-x1[0])/(num_mp[0]-1);
        dj = (x2[1]-x1[1])/(num_mp[1]-1);
        dk = (x2[2]-x1[2])/(num_mp[2]-1);

        #load particles on a equally spaced grid
        
        for i in np.arange(0,num_mp[0]):
            for j in np.arange(0,num_mp[1]):
                for k in np.arange(0,num_mp[2]):
                    
                    # sample random position
                    pos = np.zeros(3)
                    vel = np.zeros(3)
                    
                    pos[0] = x1[0] + i*di;
                    pos[1] = x1[1] + j*dj;
                    pos[2] = x1[2] + k*dk;

                    # shift particles on max faces back to the domain
                    if (pos[0]==x2[0]): pos[0]-=1e-4*di;
                    if (pos[1]==x2[1]): pos[1]-=1e-4*dj;
                    if (pos[2]==x2[2]): pos[2]-=1e-4*dk;

                    w = 1;
                    if (i==0 or i==num_mp[0]-1): w*=0.5
                    if (j==0 or j==num_mp[1]-1): w*=0.5
                    if (k==0 or k==num_mp[2]-1): w*=0.5

                    #set initial velocity
                    vel[0] = 0.0
                    vel[1] = 0.0
                    vel[2] = 0.0

                    self.addParticle(pos,vel,mpw*w)
                    
    
    def scatter_den(self, lc, value):
        """
        scatters scalar value onto a field at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        ----------
        """
        
        if lc [0] < 0 or \
             lc [0] >=self.worldObj.ni-1 or \
             lc [1] < 0 or \
             lc [1] >=self.worldObj.nj-1 or \
             lc [2] < 0 or \
             lc [2] >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = int(lc[0])
        di = lc[0]-i

        j  = int(lc[1])
        dj = lc[1]-j

        k  = int(lc[2])
        dk = lc[2]-k

        self.den[i][j][k]      += value*(1-di)*(1-dj)*(1-dk)
        self.den[i+1][j][k]    += value*(di)*(1-dj)*(1-dk)
        self.den[i+1][j+1][k]  += value*(di)*(dj)*(1-dk)
        self.den[i][j+1][k]    += value*(1-di)*(dj)*(1-dk)
        self.den[i][j][k+1]    += value*(1-di)*(1-dj)*(dk)
        self.den[i+1][j][k+1]  += value*(di)*(1-dj)*(dk)
        self.den[i+1][j+1][k+1]+= value*(di)*(dj)*(dk)
        self.den[i][j+1][k+1]  += value*(1-di)*(dj)*(dk)
        
    def computeNumberDensity(self):
        """
        Compute particle number density

        Parameters
        ----------
        
        ----------
        """
        self.den = np.zeros((self.worldObj.ni, self.worldObj.nj, self.worldObj.nk))
        
        for particle in self.particleList:
            lc = self.worldObj.XtoL(particle.pos)
            self.scatter_den(lc, self.mpw)
            
        self.den = self.den / self.worldObj.node_vol
        
    
    def gather_ef(self, lc):
        """
        gathers field value at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        data : numpy.ndarray
            electric field array
        ----------
        """
        
        if lc [0] < 0 or \
             lc [0] >=self.worldObj.ni-1 or \
             lc [1] < 0 or \
             lc [1] >=self.worldObj.nj-1 or \
             lc [2] < 0 or \
             lc [2] >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = int(lc[0])
        di = lc[0]-i

        j  = int(lc[1])
        dj = lc[1]-j

        k  = int(lc[2])
        dk = lc[2]-k

        # gather electric field onto particle position
        
        ef_x  = self.worldObj.ef[i][j][k][0]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][0]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][0]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][0]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][0]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][0]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][0]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][0]*(1-di)*(dj)*(dk)
        
        ef_y  = self.worldObj.ef[i][j][k][1]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][1]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][1]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][1]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][1]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][1]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][1]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][1]*(1-di)*(dj)*(dk)
        
        ef_z  = self.worldObj.ef[i][j][k][2]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][2]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][2]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][2]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][2]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][2]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][2]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][2]*(1-di)*(dj)*(dk)
        
        ef_part = np.array([ef_x, ef_y, ef_z])
        
        return ef_part
    
    
    def advance(self):
        #get the time step
        dt = self.worldObj.dt

        #save mesh bounds
        x0 = self.worldObj.x0
        xm = self.worldObj.xm

        # loop over all particles
        for particle in self.particleList:
        
            #get logical coordinate of particle's position
            lc = self.worldObj.XtoL(particle.pos)

            #electric field at particle position
            ef_part = self.gather_ef(lc)

            #update velocity from F=qE
            particle.vel += ef_part*(dt*self.charge/self.mass)

            #update position from v=dx/dt
            particle.pos += particle.vel*dt

            #did this particle leave the domain? reflect back
            for i in np.arange(3):
            
                if particle.pos[i] < x0[i]:
                    particle.pos[i]=2*x0[i]-particle.pos[i]
                    particle.vel[i]*=-1.0
                    
                elif particle.pos[i] >= xm[i]:
                    particle.pos[i]=2*xm[i]-particle.pos[i]
                    particle.vel[i]*=-1.0

In [35]:
world=World(20,20,20)
world.setTime(2E-10,100)
world.setExtents(-0.1, -0.1, 0.0 , 0.1, 0.1, 0.2)

species1 = Species("O+", 16*world.AMU, world.QE, world)
species2 = Species("e-", world.ME, -1.0*world.QE, world)

world.addSpeciesList([species1,species2])

species1.loadParticlesBoxQS(world.x0, world.xm, 1E11, [41,41,41])
species2.loadParticlesBoxQS(world.x0, world.xc, 1E11, [21,21,21])

species1.computeNumberDensity()
species2.computeNumberDensity()

world.computeChargeDensity()

world.potentialSolver4(1.8,10000, 1E-3)

world.efSolver2()

start1 = time.clock()

for i in range(1,20):
    species1.advance()
    species2.advance()

    species1.computeNumberDensity()
    species2.computeNumberDensity()

    world.computeChargeDensity()

    world.potentialSolver4(1.8, 10000, 1E-4)

    world.efSolver2()

    print("ts = "+str(i)+", nO+: "+str(len(species1.particleList))+", ne-: "+str(len(species2.particleList)))
    
end1 = time.clock()
print("Time: "+str(end1-start1)+" seconds")

/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


ts = 1, nO+: 68921, ne-: 9261
ts = 2, nO+: 68921, ne-: 9261
ts = 3, nO+: 68921, ne-: 9261
ts = 4, nO+: 68921, ne-: 9261
ts = 5, nO+: 68921, ne-: 9261
ts = 6, nO+: 68921, ne-: 9261
ts = 7, nO+: 68921, ne-: 9261
ts = 8, nO+: 68921, ne-: 9261
ts = 9, nO+: 68921, ne-: 9261
ts = 10, nO+: 68921, ne-: 9261
ts = 11, nO+: 68921, ne-: 9261
ts = 12, nO+: 68921, ne-: 9261
ts = 13, nO+: 68921, ne-: 9261
ts = 14, nO+: 68921, ne-: 9261
ts = 15, nO+: 68921, ne-: 9261
ts = 16, nO+: 68921, ne-: 9261
ts = 17, nO+: 68921, ne-: 9261
ts = 18, nO+: 68921, ne-: 9261
ts = 19, nO+: 68921, ne-: 9261
Time: 133.3207930000001 seconds


/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [36]:
world1=World(21,21,21)
world1.setTime(2E-10,100)
world1.setExtents(-0.1, -0.1, 0.0 , 0.1, 0.1, 0.2)

species1 = Species("O+", 16*world.AMU, world.QE, world)
species2 = Species("e-", world.ME, -1.0*world.QE, world)

world1.addSpeciesList([species1,species2])

species1.loadParticlesBoxQS(world.x0, world.xm, 1E11, [41,41,41])
species2.loadParticlesBoxQS(world.x0, world.xc, 1E11, [21,21,21])

species1.computeNumberDensity()
species2.computeNumberDensity()

world1.computeChargeDensity()

world1.potentialSolver5(1.8,10000, 1E-3)

world1.efSolver2()

start2 = time.clock()

for i in range(1,20):
    species1.advance()
    species2.advance()

    species1.computeNumberDensity()
    species2.computeNumberDensity()

    world1.computeChargeDensity()

    world1.potentialSolver5(1.8, 10000, 1E-4)

    world1.efSolver2()

    print("ts = "+str(i)+", nO+: "+str(len(species1.particleList))+", ne-: "+str(len(species2.particleList)))
    
end2 = time.clock()
print("Time: "+str(end2-start2)+" seconds")

/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


ts = 1, nO+: 68921, ne-: 9261
ts = 2, nO+: 68921, ne-: 9261
ts = 3, nO+: 68921, ne-: 9261
ts = 4, nO+: 68921, ne-: 9261
ts = 5, nO+: 68921, ne-: 9261
ts = 6, nO+: 68921, ne-: 9261
ts = 7, nO+: 68921, ne-: 9261
ts = 8, nO+: 68921, ne-: 9261
ts = 9, nO+: 68921, ne-: 9261
ts = 10, nO+: 68921, ne-: 9261
ts = 11, nO+: 68921, ne-: 9261
ts = 12, nO+: 68921, ne-: 9261
ts = 13, nO+: 68921, ne-: 9261
ts = 14, nO+: 68921, ne-: 9261
ts = 15, nO+: 68921, ne-: 9261
ts = 16, nO+: 68921, ne-: 9261
ts = 17, nO+: 68921, ne-: 9261
ts = 18, nO+: 68921, ne-: 9261
ts = 19, nO+: 68921, ne-: 9261
Time: 128.10062800000003 seconds


/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [43]:
world1=World(21,21,21)
world1.setTime(2E-10,100)
world1.setExtents(-0.1, -0.1, 0.0 , 0.1, 0.1, 0.2)

species1 = Species("O+", 16*world.AMU, world.QE, world)
species2 = Species("e-", world.ME, -1.0*world.QE, world)

world1.addSpeciesList([species1,species2])

species1.loadParticlesBoxQS(world.x0, world.xm, 1E11, [41,41,41])
species2.loadParticlesBoxQS(world.x0, world.xc, 1E11, [21,21,21])

species1.computeNumberDensity()
species2.computeNumberDensity()

world1.computeChargeDensity()

world1.potentialSolver5(1.7,10000, 1E-3)

world1.efSolver2()

start2 = time.clock()

for i in range(1,20):
    species1.advance()
    species2.advance()

    species1.computeNumberDensity()
    species2.computeNumberDensity()

    world1.computeChargeDensity()

    world1.potentialSolver5(1.7, 10000, 1E-4)

    world1.efSolver2()

    print("ts = "+str(i)+", nO+: "+str(len(species1.particleList))+", ne-: "+str(len(species2.particleList)))
    
end2 = time.clock()
print("Time: "+str(end2-start2)+" seconds")

/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


ts = 1, nO+: 68921, ne-: 9261
ts = 2, nO+: 68921, ne-: 9261
ts = 3, nO+: 68921, ne-: 9261
ts = 4, nO+: 68921, ne-: 9261
ts = 5, nO+: 68921, ne-: 9261
ts = 6, nO+: 68921, ne-: 9261
ts = 7, nO+: 68921, ne-: 9261
ts = 8, nO+: 68921, ne-: 9261
ts = 9, nO+: 68921, ne-: 9261
ts = 10, nO+: 68921, ne-: 9261
ts = 11, nO+: 68921, ne-: 9261
ts = 12, nO+: 68921, ne-: 9261
ts = 13, nO+: 68921, ne-: 9261
ts = 14, nO+: 68921, ne-: 9261
ts = 15, nO+: 68921, ne-: 9261
ts = 16, nO+: 68921, ne-: 9261
ts = 17, nO+: 68921, ne-: 9261
ts = 18, nO+: 68921, ne-: 9261
ts = 19, nO+: 68921, ne-: 9261
Time: 126.5972670000001 seconds


/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [44]:
world1=World(21,21,21)
world1.setTime(2E-10,100)
world1.setExtents(-0.1, -0.1, 0.0 , 0.1, 0.1, 0.2)

species1 = Species("O+", 16*world.AMU, world.QE, world)
species2 = Species("e-", world.ME, -1.0*world.QE, world)

world1.addSpeciesList([species1,species2])

species1.loadParticlesBoxQS(world.x0, world.xm, 1E11, [41,41,41])
species2.loadParticlesBoxQS(world.x0, world.xc, 1E11, [21,21,21])

species1.computeNumberDensity()
species2.computeNumberDensity()

world1.computeChargeDensity()

world1.potentialSolver5(1.6,10000, 1E-3)

world1.efSolver2()

start2 = time.clock()

for i in range(1,20):
    species1.advance()
    species2.advance()

    species1.computeNumberDensity()
    species2.computeNumberDensity()

    world1.computeChargeDensity()

    world1.potentialSolver5(1.6, 10000, 1E-4)

    world1.efSolver2()

    print("ts = "+str(i)+", nO+: "+str(len(species1.particleList))+", ne-: "+str(len(species2.particleList)))
    
end2 = time.clock()
print("Time: "+str(end2-start2)+" seconds")

/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


ts = 1, nO+: 68921, ne-: 9261
ts = 2, nO+: 68921, ne-: 9261
ts = 3, nO+: 68921, ne-: 9261
ts = 4, nO+: 68921, ne-: 9261
ts = 5, nO+: 68921, ne-: 9261
ts = 6, nO+: 68921, ne-: 9261
ts = 7, nO+: 68921, ne-: 9261
ts = 8, nO+: 68921, ne-: 9261
ts = 9, nO+: 68921, ne-: 9261
ts = 10, nO+: 68921, ne-: 9261
ts = 11, nO+: 68921, ne-: 9261
ts = 12, nO+: 68921, ne-: 9261
ts = 13, nO+: 68921, ne-: 9261
ts = 14, nO+: 68921, ne-: 9261
ts = 15, nO+: 68921, ne-: 9261
ts = 16, nO+: 68921, ne-: 9261
ts = 17, nO+: 68921, ne-: 9261
ts = 18, nO+: 68921, ne-: 9261
ts = 19, nO+: 68921, ne-: 9261
Time: 127.05477900000005 seconds


/home/athul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [5]:
world=World(20,20,20)
world.setTime(2E-10,100)
world.setExtents(-0.1, -0.1, 0.0 , 0.1, 0.1, 0.2)

species1 = Species("O+", 16*world.AMU, world.QE, world)
species2 = Species("e-", world.ME, -1.0*world.QE, world)

world.addSpeciesList([species1,species2])

species1.loadParticlesBoxQS(world.x0, world.xm, 1E11, [41,41,41])
species2.loadParticlesBoxQS(world.x0, world.xc, 1E11, [21,21,21])

species1.computeNumberDensity()
species2.computeNumberDensity()

world.computeChargeDensity()

world.potentialSolver2(1.8,10000, 1E-3)

world.efSolver2()

start1 = time.clock()

for i in range(1,20):
    species1.advance()
    species2.advance()

    species1.computeNumberDensity()
    species2.computeNumberDensity()

    world.computeChargeDensity()

    world.potentialSolver2(1.8, 10000, 1E-4)

    world.efSolver2()

    print("ts = "+str(i)+", nO+: "+str(len(species1.particleList))+", ne-: "+str(len(species2.particleList)))
    
end1 = time.clock()
print("Time: "+str(end1-start1)+" seconds")

<ipython-input-2-e3f6a017811c>:263: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "potentialSolver2" failed type inference due to: Untyped global name 'str': cannot determine Numba type of <class 'type'>

File "<ipython-input-2-e3f6a017811c>", line 370:
    def potentialSolver2(self, w, max_it, tol):
        <source elided>
        if (converged==False):
            print("Gauss-Seidel failed to converge, L2 = "+str(L2))
            ^

  @jit
<ipython-input-2-e3f6a017811c>:263: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "potentialSolver2" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-2-e3f6a017811c>", line 299:
    def potentialSolver2(self, w, max_it, tol):
        <source elided>
        # solve potential
        for it in np.arange(1,max_it+1):
        ^

  @jit


LoweringError: Failed in object mode pipeline (step: object mode frontend)
Failed in object mode pipeline (step: object mode backend)
[1m[1m(<class 'numba.ir.StaticSetItem'>, x[(slice(1, -1, None), slice(1, -1, None), slice(1, -1, None))] = $0.77)
[1m
File "<ipython-input-2-e3f6a017811c>", line 290:[0m
[1m    def potentialSolver2(self, w, max_it, tol):
        <source elided>
        # Step 2: populate all non-boundary cells with running numbers from 1 to (n-2)^2
[1m        x[1:-1,1:-1,1:-1] = np.arange(1,(self.ni-2)*(self.nj-2)*(self.nk-2)+1).reshape(self.ni-2,self.nj-2,self.nk-2)
[0m        [1m^[0m[0m
[0m
[0m[1m[1] During: lowering "x[(slice(1, -1, None), slice(1, -1, None), slice(1, -1, None))] = $0.77" at <ipython-input-2-e3f6a017811c> (290)[0m
-------------------------------------------------------------------------------
This should not have happened, a problem has occurred in Numba's internals.

Please report the error message and traceback, along with a minimal re